<a href="https://colab.research.google.com/github/KishenPatel97/Book_Summarization/blob/main/bleu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import nltk
from nltk.translate import bleu
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
nltk.download('punkt')
from nltk import word_tokenize
# from summarize import processBook

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [77]:
hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to help students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and vocabulary at the bottom of the page, comment for a chance to be mentioned on cnn student news. you must be a teacher or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests students' knowledge of even ts in the news"

hy1 = "This text is just for testing purposes"

ref1 = "The article is just meant for evaluation"
# Requires text in a tokenized format
score1 = bleu([word_tokenize(reference)], word_tokenize(hypothesis))
score1_2 = corpus_bleu([[word_tokenize(reference)]], [word_tokenize(hypothesis)])
print(score1)
print(score1_2)

chencherry = SmoothingFunction()
score2 = bleu([word_tokenize(ref1)], word_tokenize(hyp1), smoothing_function = chencherry.method1)
print(score2)

0.17607804141687589
0.17607804141687589
0.07730551756939454


In [14]:
def processBook(fname):
    """ Function to process .txt books, esp. from Gutenberg.
    PARAMS: fname (str) - filepath to book to be processed into a string
    RETURNS: (str) - the string object containing the text
    """
    book = open(fname)      # open book file
    book_lines = []         # initialize storage
    for line in book.readlines():       # iterate through each line
        book_lines.append(line.strip())     # process each line

    return " ".join(book_lines)         # return processed lines

In [115]:
def evaluateSummary(controltext1, controltext2, summarytext):
    """Helper function to run summarization experiments
    PARAMS: controltext1 (str) - path to Cliff's Notes summary
            controltext2 (str) - path to Grade Saver summary
            summarytext (str) - path to generated summary to evaluate
    """
    control1 = processBook(controltext1)
    control2 = processBook(controltext2)
    summary = processBook(summarytext)

    chencherry = SmoothingFunction()
    # Requires text in a tokenized format
    # Calculate the upper bound (Bleu of 2 manually created summaries)
    ub_scores = bleu([word_tokenize(control1)], word_tokenize(control2), weights = (1, 0, 0, 0), smoothing_function = chencherry.method1)
    ub_scores_bi = bleu([word_tokenize(control1)], word_tokenize(control2), weights = (0, 1, 0, 0), smoothing_function = chencherry.method1)
    ub_scores_cumal = bleu([word_tokenize(control1)], word_tokenize(control2), smoothing_function = chencherry.method1)

    ub_scores2 = bleu([word_tokenize(control2)], word_tokenize(control1), weights = (1, 0, 0, 0), smoothing_function = chencherry.method1)
    ub_scores2_bi = bleu([word_tokenize(control2)], word_tokenize(control1), weights = (0, 1, 0, 0), smoothing_function = chencherry.method1)
    ub_scores2_cumal = bleu([word_tokenize(control2)], word_tokenize(control1), smoothing_function = chencherry.method1)

    sum_scores_cn = bleu([word_tokenize(summary)], word_tokenize(control1), weights = (1, 0, 0, 0), smoothing_function = chencherry.method1)
    sum_scores_cn_bi = bleu([word_tokenize(summary)], word_tokenize(control1), weights = (0, 1, 0, 0), smoothing_function = chencherry.method1)
    sum_scores_cn_cumal = bleu([word_tokenize(summary)], word_tokenize(control1), smoothing_function = chencherry.method1)

    sum_scores_gs = bleu([word_tokenize(summary)], word_tokenize(control2), weights = (1, 0, 0, 0), smoothing_function = chencherry.method1)
    sum_scores_gs_bi = bleu([word_tokenize(summary)], word_tokenize(control2), weights = (0, 1, 0, 0), smoothing_function = chencherry.method1)
    sum_scores_gs_cumal = bleu([word_tokenize(summary)], word_tokenize(control2), smoothing_function = chencherry.method1)


    #print("Scores for {}".format(controltext1[18:]))
    print(f"Scores for {controltext1}")
    print("Upper Bound:")
    print(f'unigram:    {ub_scores}')
    print(f'bigram:     {ub_scores_bi}')
    print(f'cumalitive  {ub_scores_cumal}')
    print(f'\nunigram:    {ub_scores2}')
    print(f'bigram:     {ub_scores2_bi}')
    print(f'cumalitive  {ub_scores2_cumal}')
    print("\nCliff's Notes:")
    print(f'unigram:    {sum_scores_cn}')
    print(f'bigram:     {sum_scores_cn_bi}')
    print(f'cumalitive: {sum_scores_cn_cumal}')
    print("\nGradeSaver:")
    print(f'unigram:    {sum_scores_gs}')
    print(f'bigram:     {sum_scores_gs_bi}')
    print(f'cumalitive: {sum_scores_gs_cumal}')

In [116]:
evaluateSummary("cn_alice.txt",
                "gs_alice.txt",
                "whole_summary_alice.txt")

Scores for cn_alice.txt
Upper Bound:
unigram:    0.08303134720918029
bigram:     0.032459204484857264
cumalitive  0.01419038319425634

unigram:    0.2351590685470646
bigram:     0.09186351706036745
cumalitive  0.04014594269451768

Cliff's Notes:
unigram:    0.1807149885208265
bigram:     0.03608923884514437
cumalitive: 0.009155717540088395

GradeSaver:
unigram:    0.3405887852289164
bigram:     0.06650345681735587
cumalitive: 0.02163978358960259


In [122]:
evaluateSummary("cn_david.txt",
                "gs_david.txt",
                "whole_summary_david.txt")

Scores for cn_david.txt
Upper Bound:
unigram:    0.3392307692307692
bigram:     0.13086989992301773
cumalitive  0.056846519709643065

unigram:    0.22932578989714053
bigram:     0.08854956898635996
cumalitive  0.038480971039978065

Cliff's Notes:
unigram:    0.2730039072978139
bigram:     0.027222154315925295
cumalitive: 0.005815222919923571

GradeSaver:
unigram:    0.25538461538461543
bigram:     0.03233256351039261
cumalitive: 0.00559644898758214


In [123]:
evaluateSummary("cn_expectations.txt",
                "gs_expectations.txt",
                "whole_summary_expectations.txt")

Scores for cn_expectations.txt
Upper Bound:
unigram:    0.46613545816733065
bigram:     0.18768115942028984
cumalitive  0.08229812443984479

unigram:    0.40850835159465515
bigram:     0.16451470490322925
cumalitive  0.07214756792939755

Cliff's Notes:
unigram:    0.14767441860465116
bigram:     0.01687027341477603
cumalitive: 0.0030315063625104615

GradeSaver:
unigram:    0.10358565737051793
bigram:     0.01594202898550724
cumalitive: 0.003052385012636237


In [117]:
evaluateSummary("cn_frank.txt",
                "gs_frank.txt",
                "whole_summary_frank.txt")

Scores for cn_frank.txt
Upper Bound:
unigram:    0.2972972972972973
bigram:     0.10815682739972962
cumalitive  0.041622924169551496

unigram:    0.17754126774891524
bigram:     0.06462988085014275
cumalitive  0.02487986089601523

Cliff's Notes:
unigram:    0.28802154657946133
bigram:     0.038299639348021566
cumalitive: 0.005830722513642557

GradeSaver:
unigram:    0.3
bigram:     0.060387561964849026
cumalitive: 0.014188404137624445


In [124]:
evaluateSummary("cn_huckfin.txt",
                "gs_huckfin.txt",
                "whole_summary_huckfin.txt")

Scores for cn_huckfin.txt
Upper Bound:
unigram:    0.2803921568627451
bigram:     0.11378126532614026
cumalitive  0.05191206501684533

unigram:    0.1484987615856758
bigram:     0.06030670128575214
cumalitive  0.027525336504052856

Cliff's Notes:
unigram:    0.2509505703422053
bigram:     0.029187817258883243
cumalitive: 0.008295892765391681

GradeSaver:
unigram:    0.14215686274509803
bigram:     0.017165277096615986
cumalitive: 0.00276889100269416


In [118]:
evaluateSummary("cn_pride.txt",
                "gs_pride.txt",
                "whole_summary_pride.txt")

Scores for cn_pride.txt
Upper Bound:
unigram:    0.3231707317073171
bigram:     0.14449104079298514
cumalitive  0.06221505030388924

unigram:    0.18140364266272244
bigram:     0.08115244351259103
cumalitive  0.03495262654841914

Cliff's Notes:
unigram:    0.3282732447817837
bigram:     0.03513770180436847
cumalitive: 0.005683260935204823

GradeSaver:
unigram:    0.1676829268292683
bigram:     0.025162028211971027
cumalitive: 0.004185778054362778


In [125]:
evaluateSummary("cn_scarlet.txt",
                "gs_scarlet.txt",
                "whole_summary_scarlet.txt")

Scores for cn_scarlet.txt
Upper Bound:
unigram:    0.4584786624933773
bigram:     0.1311605624799302
cumalitive  0.048563373167915966

unigram:    0.47834456207892206
bigram:     0.13680154142581888
cumalitive  0.050644147959759196

Cliff's Notes:
unigram:    0.2983638113570741
bigram:     0.04238921001926782
cumalitive: 0.006965835885137323

GradeSaver:
unigram:    0.31596701417050305
bigram:     0.0465969856489687
cumalitive: 0.0068659691664485845


In [119]:
evaluateSummary("cn_tale.txt",
                "gs_tale.txt",
                "whole_summary_tale.txt")

Scores for cn_tale.txt
Upper Bound:
unigram:    0.39036978012722634
bigram:     0.1327118069258435
cumalitive  0.06417368020060649

unigram:    0.43161634103019536
bigram:     0.14666666666666667
cumalitive  0.07090525812334236

Cliff's Notes:
unigram:    0.25754884547069273
bigram:     0.04266666666666667
cumalitive: 0.005431884961326877

GradeSaver:
unigram:    0.30169268682148603
bigram:     0.05481650131126973
cumalitive: 0.007045445958764615


In [120]:
evaluateSummary("cn_treasure.txt",
                "gs_treasure.txt",
                "whole_summary_treasure.txt")

Scores for cn_treasure.txt
Upper Bound:
unigram:    0.49544922544410486
bigram:     0.1454979131243865
cumalitive  0.0615375452620419

unigram:    0.4977738201246661
bigram:     0.14616755793226383
cumalitive  0.06181800524638476

Cliff's Notes:
unigram:    0.2653606411398041
bigram:     0.031194295900178255
cumalitive: 0.006026005363857137

GradeSaver:
unigram:    0.2595494613124388
bigram:     0.03823529411764706
cumalitive: 0.007865125359962121


In [126]:
evaluateSummary("cn_warpeace.txt",
                "gs_warpeace.txt",
                "whole_summary_warpeace.txt")

Scores for cn_warpeace.txt
Upper Bound:
unigram:    0.3752913752913753
bigram:     0.10447761194029852
cumalitive  0.030991563369445897

unigram:    0.3365555744093594
bigram:     0.09371762107043731
cumalitive  0.027803312488139758

Cliff's Notes:
unigram:    0.17038102084831053
bigram:     0.01942446043165468
cumalitive: 0.002035493017924886

GradeSaver:
unigram:    0.14498834498834498
bigram:     0.01912313432835821
cumalitive: 0.0027878065405554154
